In [1]:
pip install pyperclip

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11020 sha256=3ed7b9862ac7ac6578dca1feeb512c8da17d306d935b783336e1b1fe41d27443
  Stored in directory: c:\users\ankit\appdata\local\pip\cache\wheels\cc\ae\36\ee17d1de094fcb61e24106cb329b5103861e819f94bef5e10a
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
import datetime as dt
from IPython.display import clear_output
import breeze_client as bc
import yfinance as yf
import time



In [10]:
df =pd.read_csv('trading_api/2024-06-14_NSE.csv')



df['datetime'] = df['datetime'].apply(lambda x : dt.datetime.fromtimestamp(x/1000))
df

,datetime,ltp,qty
0,2024-06-14 11:43:45.389,95.50,250
1,2024-06-14 11:43:45.882,95.60,25
2,2024-06-14 11:43:46.581,95.30,25
3,2024-06-14 11:43:46.628,95.45,25
4,2024-06-14 11:43:47.570,95.30,250
...,...,...,...
25364,2024-06-14 15:29:58.967,89.00,100
25365,2024-06-14 15:29:59.460,88.00,25
25366,2024-06-14 15:29:59.965,89.00,25
25367,2024-06-14 15:29:59.978,89.00,100


In [11]:
df.set_index('datetime',inplace=True)
df

,ltp,qty
datetime,,
2024-06-14 11:43:45.389,95.50,250
2024-06-14 11:43:45.882,95.60,25
2024-06-14 11:43:46.581,95.30,25
2024-06-14 11:43:46.628,95.45,25
2024-06-14 11:43:47.570,95.30,250
...,...,...
2024-06-14 15:29:58.967,89.00,100
2024-06-14 15:29:59.460,88.00,25
2024-06-14 15:29:59.965,89.00,25


In [14]:

resampled_df = df['ltp'].resample('1S').ohlc()
resampled_df['volume'] = df['qty'].resample('1S').sum()

resampled_df

,open,high,low,close,volume
datetime,,,,,
2024-06-14 11:43:45,95.50,95.60,95.50,95.60,275
2024-06-14 11:43:46,95.30,95.45,95.30,95.45,50
2024-06-14 11:43:47,95.30,95.30,95.25,95.25,300
2024-06-14 11:43:48,95.30,95.30,95.20,95.20,75
2024-06-14 11:43:49,95.25,95.25,95.25,95.25,100
...,...,...,...,...,...
2024-06-14 15:29:55,88.20,88.20,88.20,88.20,250
2024-06-14 15:29:56,89.00,89.00,89.00,89.00,50
2024-06-14 15:29:57,88.20,88.20,88.20,88.20,450


In [17]:
resampled_df.rename(columns={'high':'High','close':'Close','low':'Low','open':'Open','volume':'Volume'},inplace=True)

In [4]:
help(bt.lib.resample_apply)

Help on function resample_apply in module backtesting.lib:

resample_apply(rule: str, func: Optional[Callable[..., Sequence]], series: Union[pandas.core.series.Series, pandas.core.frame.DataFrame, backtesting._util._Array], *args, agg: Union[str, dict] = None, **kwargs)
    Apply `func` (such as an indicator) to `series`, resampled to
    a time frame specified by `rule`. When called from inside
    `backtesting.backtesting.Strategy.init`,
    the result (returned) series will be automatically wrapped in
    `backtesting.backtesting.Strategy.I`
    wrapper method.
    
    `rule` is a valid [Pandas offset string] indicating
    a time frame to resample `series` to.
    
    [Pandas offset string]: http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    `func` is the indicator function to apply on the resampled series.
    
    `series` is a data series (or array), such as any of the
    `backtesting.backtesting.Strategy.data` series. Due to pandas
    resamp

In [7]:
def supertrend(data,length=10,multiplier=3):

        super = ta.supertrend(data.High.s,data.Low.s,data.Close.s,length,multiplier) 

        return super.to_numpy().T[0]

def resample_supertrend(High,Low,Close,length=10,multiplier=3):

        super = ta.supertrend(High,Low,Close,length,multiplier) 

        return super.to_numpy().T[0]



def atr(dfpl,length):

        atr = ta.atr(dfpl.High.s, dfpl.Low.s, dfpl.Close.s, length)

        return atr.to_numpy()





In [8]:



class SuperTrend(bt.Strategy):

    multiplier = 3
    period = 9
    atr_length = 7

    
    def init(self):
        
        self.super = self.I(supertrend,self.data,self.period,self.multiplier) 
        self.ATR = self.I(atr,self.data,self.atr_length)


        self.super_1m = bt.lib.resample_apply('1T',resample_supertrend,self.data.High.s,self.data.Low.s,self.data.Close.s)

    
    def next(self):
        slatr = 1.2*self.ATR[-1]
        TPSLRatio = 1.5

        current_price = self.data.Close[-1]


        n_current_price = self.data.Close[-1]
        last_price = self.data.Close[-2]
        current_trend = self.super[-1]
        last_trend = self.super[-2]
        
        #buy signal
        if (last_trend > last_price) and (current_trend < n_current_price):
            self.position.close()

            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1,)

            
            
                

        if (last_trend < last_price) and (current_trend > n_current_price):
            self.position.close()
            
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1,)

                
        

        


            


        
    

In [63]:



class SuperTrend(bt.Strategy):

    length = 400
    upper= 55
    lower = 45

    
    def init(self):
        self.rsi = self.I(ta.rsi,self.data.Close.s,self.length)
        self.minute_rsi = bt.lib.resample_apply('1T', ta.rsi, self.data.Close, self.length)


        

    
    def next(self):

        if self.rsi > self.upper:
            self.sell()

        if self.rsi < self.lower :
            self.buy()
        
        
        

                
        

        


            


        
    

In [64]:
eng = bt.Backtest(df,SuperTrend,cash=25000)

C:\Users\Ankit\AppData\Local\Temp\ipykernel_8840\2278407272.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  eng = bt.Backtest(df,SuperTrend,cash=25000)


In [65]:
stats = eng.run()
stats

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [51]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1291', ...)

In [59]:
def optim_func(series):

    if series['# Trades'] > 10:
        return series['Return [%]']
    
    


    else:
        return -1000000

opt_stats,heatmap = eng.optimize(
    length = range(300,500),
    upper = range(50,70),
    lower = range(30,50),
    #period = range(3,50),
    
    maximize = optim_func,
    
    return_heatmap = True,
    max_tries = 1000

)
opt_stats

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 1037 configurations.
  output = _optimize_grid()


Start                                     0.0
End                                   21151.0
Duration                              21151.0
Exposure Time [%]                   98.425681
Equity Final [$]                    117320.35
Equity Peak [$]                     129360.35
Return [%]                           369.2814
Buy & Hold Return [%]               237.22334
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -27.142776
Avg. Drawdown [%]                   -2.655967
Max. Drawdown Duration                 3398.0
Avg. Drawdown Duration             127.925466
# Trades                                 12.0
Win Rate [%]                            100.0
Best Trade [%]                     171.240602
Worst Trade [%]                     11.185309
Avg. Trade [%]                    

In [61]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2877', ...)

In [62]:
bt.lib.plot_heatmaps(heatmap,agg='mean')

GridPlot(id='p3038', ...)

In [ ]:
class VWAP(bt.Strategy):

    length = 7

    def __init__(self):
        self.vwap = ta.vwap(self.data.High,self.data.Low,self.data.Close,self.data.Volume)

    def next(self):
        pass

In [19]:
resampled_df['vwap'] = ta.vwap(resampled_df.High,resampled_df.Low,resampled_df.Close,resampled_df.Volume)
resampled_df

,Open,High,Low,Close,Volume,vwap
datetime,,,,,,
2024-06-14 11:43:45,95.50,95.60,95.50,95.60,275,95.566667
2024-06-14 11:43:46,95.30,95.45,95.30,95.45,50,95.541026
2024-06-14 11:43:47,95.30,95.30,95.25,95.25,300,95.409333
2024-06-14 11:43:48,95.30,95.30,95.20,95.20,75,95.390476
2024-06-14 11:43:49,95.25,95.25,95.25,95.25,100,95.372917
...,...,...,...,...,...,...
2024-06-14 15:29:55,88.20,88.20,88.20,88.20,250,88.087914
2024-06-14 15:29:56,89.00,89.00,89.00,89.00,50,88.087940
2024-06-14 15:29:57,88.20,88.20,88.20,88.20,450,88.087969
